In [8]:
import pandas as pd
import numpy as np
from openai import OpenAI
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import os
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
import openpyxl

load_dotenv()

df = pd.read_csv('국토안전관리원_민원데이터_20250101.csv', encoding='cp949')

In [2]:
train_df = pd.read_csv('train_data.csv', encoding='utf-8')

In [3]:
test_df = pd.read_csv('test_data.csv', encoding='utf-8')

In [9]:
train_df.to_excel('train_data.xlsx')
test_df.to_excel('test_data.xlsx')

In [6]:
EMBED_MODEL_ID = "Qwen/Qwen3-Embedding-0.6B"
EMBED_DIM = 1024  # Qwen은 1024차원 (중요!)
COLLECTION_NAME = "complaints"  # 원하면 한글로 해도 됨

model = SentenceTransformer(EMBED_MODEL_ID, trust_remote_code=True)

client_qd = QdrantClient(url="http://localhost:6333")

print("Qdrant 연결 설정 완료!")

Qdrant 연결 설정 완료!


In [10]:
def ensure_collection():
    print("🔍 Checking collection existence...")
    exists = client_qd.collection_exists(collection_name=COLLECTION_NAME)
    print(f"→ Exists? {exists}")

    if not exists:
        client_qd.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(size=EMBED_DIM, distance=Distance.COSINE),
        )
        print(f"✅ Created collection: {COLLECTION_NAME}")
    else:
        print(f"⚙️ Collection already exists: {COLLECTION_NAME}")
        
ensure_collection()

🔍 Checking collection existence...
→ Exists? False
✅ Created collection: complaints


In [12]:
def embed_batch(texts):
    # 1. 전처리 (매우 중요: 빈 문자열이 API나 모델에 들어가면 에러 발생 가능)
    cleaned = [t.strip() for t in texts if isinstance(t, str) and t.strip()]
    
    if not cleaned:
        return [], []

    # 2. 모델 실행 (Qwen/SentenceTransformer 사용 시)
    # normalize_embeddings=True는 코사인 유사도 계산 시 성능을 높여줍니다.
    embeddings = model.encode(cleaned, normalize_embeddings=True)
    
    return embeddings.tolist(), cleaned

In [16]:
# 4) 배치로 처리 (Qwen/로컬 모델 버전)
points_all = []
n = len(df)
BATCH_SIZE = 5

for start in range(0, n, BATCH_SIZE):
    end = min(start + BATCH_SIZE, n)
    batch_df = df.iloc[start:end].copy() # 원본 보존을 위해 copy 사용

    # 1. 배치 텍스트 구성
    batch_texts = [
        f"[하위분류] {row['하위분류']}\n[문의내용] {row['문의내용']}"
        for _, row in batch_df.iterrows()
    ]

    # 2. 임베딩 호출 (위에서 만든 embed_batch 함수 사용)
    # vectors: 임베딩 결과, cleaned: 전처리로 살아남은 텍스트들
    vectors, cleaned = embed_batch(batch_texts)

    # 만약 전처리 결과 모든 데이터가 비어있다면 건너뜀
    if not vectors:
        continue

    # 3. Qdrant 포인트 생성
    points = []
    
    # 주의: embed_batch에서 빈 문자열이 제거되었다면 batch_df와 vectors의 개수가 다를 수 있습니다.
    # 여기서는 데이터 정제가 수행된 후에도 순서가 유지된다고 가정하고 
    # batch_df를 vectors의 개수에 맞춰서 루프를 돕니다.
    for i, (idx, row) in enumerate(batch_df.iterrows()):
        # 만약 embed_batch 내에서 필터링이 일어났을 경우를 대비한 안전장치
        if i >= len(vectors):
            break
            
        p = PointStruct(
            id=int(idx),
            vector=vectors[i],  # 리스트 형태의 벡터
            payload={
                "하위분류": row["하위분류"],
                "문의내용": row["문의내용"],
                "답변부서": row["답변부서"],
                "original_text": cleaned[i] # 정제된 텍스트도 저장하고 싶을 경우
            },
        )
        points.append(p)

    # 4. 배치 업서트
    if points:
        client_qd.upsert(
            collection_name="complaints",
            points=points,
        )

    print(f"{end}/{n} 까지 업로드 완료 (현재 배치 {len(points)}개 처리)")

print("전체 업로드 완료!")

5/716 까지 업로드 완료 (현재 배치 5개 처리)
10/716 까지 업로드 완료 (현재 배치 5개 처리)
15/716 까지 업로드 완료 (현재 배치 5개 처리)
20/716 까지 업로드 완료 (현재 배치 5개 처리)
25/716 까지 업로드 완료 (현재 배치 5개 처리)
30/716 까지 업로드 완료 (현재 배치 5개 처리)
35/716 까지 업로드 완료 (현재 배치 5개 처리)
40/716 까지 업로드 완료 (현재 배치 5개 처리)
45/716 까지 업로드 완료 (현재 배치 5개 처리)
50/716 까지 업로드 완료 (현재 배치 5개 처리)
55/716 까지 업로드 완료 (현재 배치 5개 처리)
60/716 까지 업로드 완료 (현재 배치 5개 처리)
65/716 까지 업로드 완료 (현재 배치 5개 처리)
70/716 까지 업로드 완료 (현재 배치 5개 처리)
75/716 까지 업로드 완료 (현재 배치 5개 처리)
80/716 까지 업로드 완료 (현재 배치 5개 처리)
85/716 까지 업로드 완료 (현재 배치 5개 처리)
90/716 까지 업로드 완료 (현재 배치 5개 처리)
95/716 까지 업로드 완료 (현재 배치 5개 처리)
100/716 까지 업로드 완료 (현재 배치 5개 처리)
105/716 까지 업로드 완료 (현재 배치 5개 처리)
110/716 까지 업로드 완료 (현재 배치 5개 처리)
115/716 까지 업로드 완료 (현재 배치 5개 처리)
120/716 까지 업로드 완료 (현재 배치 5개 처리)
125/716 까지 업로드 완료 (현재 배치 5개 처리)
130/716 까지 업로드 완료 (현재 배치 5개 처리)
135/716 까지 업로드 완료 (현재 배치 5개 처리)
140/716 까지 업로드 완료 (현재 배치 5개 처리)
145/716 까지 업로드 완료 (현재 배치 5개 처리)
150/716 까지 업로드 완료 (현재 배치 5개 처리)
155/716 까지 업로드 완료 (현재 배치 5개 처리)
160/716 까지 업로드 완료 (현재 배치 5개 

In [11]:
results = rag_answer("집가고싶다")

In [9]:
results.points[0].payload.get('답변부서')

'지진안전관리실'

In [12]:
for point in results.points:
    print(point.payload)

{'하위분류': '시설물 내진성능평가 및 내진보강 관련 문의', '문의내용': '안녕하십니까제주개발공사에서 운영하고 있는 소규모 임대주택 내진성능평가 용역을 수행하고자 계획 중에 있습니다. 해당 주택의 내진성능평가 대가 산정방식에 대해 문의드리고자 합니다. A동(연면적 659m2), B동(327m2), C동(520m2)을 내진성능평가를 실시하고자 합니다. A동과 B동의 거리는 5.0km 이고 차량으로 16분 소요되고, B동과 C동의 거리는 2.4km 이고, 차량으로 6분 소요되는 상황입니다. (참고로 A-B-C, 지도상 일직선상에 놓여 있습니다)민원답변에서 군집시설물에 대해 찾다보니 군집시설물은 점검, 진단시 현장이동, 업무협의, 보고서 작성 및 중간보고 등의 업무 등에 대해 중복되는 부분에서 기준인원수를 조정하도록 한 부분이라고 회신된 내용을 보았습니다.(답변일자 2018. 03. 12., 시설성능연구소 정일원) 도보상 이동은 어려우나, 해당 업무를 수행함에 있어 동일한 인력으로 수행시 중복되는 인력이 존재할 것이라고 보여집니다. 군집시설물로 볼 수 있는지 여부가 궁금하여 질문을 남깁니다. 회신 부탁드립니다.', '답변부서': '지진안전관리실'}
{'하위분류': '안전관리계획서 작성 관련 문의', '문의내용': '안녕하세요 고생 많으십니다!정기안전점검 항목이지보공2회, 동바리2회, 합벽2회라고 가정하겠습니다.직접인건비에 관한 질의입니다!내업, 외업을 어떻게 나누며, 회당 비용이 천차만별인데 그냥 초급 1명 넣어놓으면 시공사 입장에서 회당 비용 적어서좋은 입장인데 관련된 근거가 있나요? 그냥 직접 인건비에 안전관리계획서 작성하시는 분 마음대로 초급 1명 넣어서 점검 회 당 비용 50만원 정도에 해도 되는 건가요? 그렇게 되면 위에 가정 한대로총 6회를 300만원에 하게 되는데 여기저기 너무 천차만별 의견도 가지가지여서 확실한 답을 듣고 싶어 질의합니다. 뚜렷한 기준이 없어 난감한 상황이 많습니다. 여러 회사의 안전관리계획서 첨부드립니다.', '답변부서': '건

In [12]:
def choose_department_with_llm(query_text: str, hits) -> str:
    """유사 민원들을 보고 LLM이 담당 부서를 한 개 고르도록"""

    examples_text = ""
    for h in hits.points:
        p = h.payload
        examples_text += (
            f"- 문의내용: {p['문의내용']}\n"
            f"  부서: {p['답변부서']}\n\n"
        )

    prompt = f"""
너는 민원 라우팅 보조 시스템이다.

아래는 과거 민원 문의내용과 실제로 배정된 담당 부서 목록이다.

[과거 민원 사례]
{examples_text}

위 사례들을 참고해서, 아래 '새 민원'에 대해 가장 적절한 담당 부서 하나를 골라라.
반드시 위 사례에서 등장한 부서명 중 하나만 고르거나, 적절한 부서가 없을 경우 '추가민원'을 출력하라.
그리고 그 이유를 작성하라.

[새 민원]
\"\"\"{query_text}\"\"\"
"""

    resp = chat_client.chat.completions.create(
        model="solar-pro2",
        messages=[{"role": "user", "content": prompt}],
    )
    department = resp.choices[0].message.content.strip()
    return department

In [13]:
question = "답변좀 씹지 마세요!!"
print(choose_department_with_llm(question, rag_answer(question)))

**결과:** 추가민원  
**이유:**  

1. **사례 분석:**  
   - 과거 민원 사례들은 모두 **시설물의 분류, 안전진단 기준, 법적 해석** 등 전문적인 기술적·행정적 질의에 관한 내용이었으며, 이에 따라 **시설성능관리실** 또는 **기획조정실**이 담당 부서로 배정되었습니다.  
   - 특히 **시설성능관리실**은 시설물안전법, 세부지침, 구조해석 등과 관련된 문의를 전담하는 부서입니다.  

2. **새 민원의 성격:**  
   - "답변좀 씹지 마세요!!"는 **민원 내용 자체가 없으며, 단순히 답변 태도에 대한 불만성 표현**입니다.  
   - 이는 특정 부서(시설성능관리실/기획조정실)의 업무 범위나 전문성과 무관한 **감정적 항의**에 해당합니다.  

3. **적절한 부서 부재:**  
   - 제공된 사례에는 **불만/항의 민원**을 전담하는 부서가 명시되지 않았습니다.  
   - 따라서 기존 사례와의 연관성이 없으며, **"추가민원"**으로 분류해 별도의 처리 프로세스(예: 고객지원팀, 민원처리센터)로 이관하는 것이 적절합니다.  

4. **예외 사항 배제:**  
   - 만약 이 민원이 특정 민원(예: 시설성능관리실의 답변)과 연결된 후속 질의였다면 해당 부서로 배정할 수 있으나, 현재 문맥상 그러한 근거가 없습니다.  

**결론:** 기술적·행정적 질의가 아닌 감정적 항의이므로, **"추가민원"**으로 분류하는 것이 가장 타당합니다.
